* Try cleaning the tweets that you have extracted in the the previous chapter. Apply the above rules and in addition to that apply the below mentioned rules as well:
    * Remove Pucnctuations. Pucntuations some times don't carry any weight. You can remove them. Try writing a regular expression to remove , from sentences. Dont remove question marks "?" or exclamatory marks as they have effect upon any sentence.
    * Remove Apostrophe's and expand the words. For example in the sentence "It's a great time to code!" the first word It's can be expanded to it is. You can do this either with regular expressions.
    * Create a list of word pattenrs for word formatting. For example gud should be substitued with good

* Calculate the polarity of a sentence and write a progam to calculatet the polarity of all the tweets that you have extracted and preprocessed in the previous questions. You progam should also include the below features:

    * Tweets have hashtags. Remove the hashtags and then find the polarity of each tweet.

    * There might be words that are not present in the sentiwordnet lexicon.
    * The program should handle these cases, by giving a zero score for such words.
    *Depending on the questions,file uploads or screenshots are necessary to show your work.